In [1]:
import tensorflow as tf
from tensorflow import keras
import pathlib
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,AveragePooling2D
from keras.layers import BatchNormalization
from keras.callbacks import LearningRateScheduler,ReduceLROnPlateau
from keras.optimizers import Adam
from keras import Model, Input
from keras.layers import Average
from typing import List
from keras.utils.version_utils import training


print(tf.config.list_physical_devices('CPU'))

c:\Users\vit\.conda\envs\HICT\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\vit\.conda\envs\HICT\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\vit\.conda\envs\HICT\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Загрузка датасета и связанных гиперпараметров

In [2]:
local_path = 'D:/Study/HICT/HICT_Patterns/'

data_dir = pathlib.Path(f'{local_path}datasets/images_center_10')

In [3]:
batch_size = 32
img_height = 20
img_width = 20
num_classes = 2

In [4]:
with tf.device('/device:CPU:0'):
  train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 5804 files belonging to 2 classes.
Using 4644 files for training.


In [5]:
with tf.device('/device:CPU:0'):
  val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 5804 files belonging to 2 classes.
Using 1160 files for validation.


In [6]:
class_names = train_ds.class_names
print(class_names)

['negative', 'positive']


In [7]:
with tf.device('/device:CPU:0'):
  AUTOTUNE = tf.data.AUTOTUNE

  train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
  val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Определение моделей и методов для работы с ними

In [8]:
def create_model_cnn(input_shape=(img_height, img_width), num_classes=num_classes):
  model = Sequential([
  keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3), name='rescaling_1_1'),
  keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Dropout(0.2),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(num_classes)
  ])
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [9]:
def create_model_cnn_2(input_shape=(img_height, img_width), num_classes=num_classes):
    model = Sequential()
    model.add(keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3), name='rescaling_2_1'))
    model.add(Conv2D(32, kernel_size = (3,3), activation='relu', input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = (5,5), strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Conv2D(64, kernel_size = (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Conv2D(128, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(num_classes, activation = "sigmoid"))
    model.compile(optimizer = 'adam' , loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
    return model

In [10]:
def create_LeNet5(input_shape=(img_height, img_width), num_classes=num_classes):
    model = Sequential()
    model.add(keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3), name='rescaling_3_1'))
    model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape, padding="same"))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))
    model.add(Flatten())
    model.add(Dense(84, activation='relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    model.compile(optimizer =  'adam' , loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
    return model

In [11]:
def train_model(model, epochs):
    with tf.device('/device:CPU:0'):
        history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs)
        return history

# Создание моделей

In [12]:
model_cnn = create_model_cnn()
model_cnn_2 = create_model_cnn_2()
model_LeNet = create_LeNet5()

# Обучение

In [13]:
history_cnn = train_model(model_cnn, 30)

Epoch 1/30
146/146 [==============================] - 3s 12ms/step - loss: 0.1310 - accuracy: 0.9472 - val_loss: 0.0227 - val_accuracy: 0.9948
Epoch 2/30
146/146 [==============================] - 1s 10ms/step - loss: 0.0276 - accuracy: 0.9929 - val_loss: 0.0232 - val_accuracy: 0.9914
Epoch 3/30
146/146 [==============================] - 2s 11ms/step - loss: 0.0214 - accuracy: 0.9938 - val_loss: 0.0228 - val_accuracy: 0.9914
Epoch 4/30
146/146 [==============================] - 2s 10ms/step - loss: 0.0206 - accuracy: 0.9953 - val_loss: 0.0157 - val_accuracy: 0.9948
Epoch 5/30
146/146 [==============================] - 1s 10ms/step - loss: 0.0171 - accuracy: 0.9974 - val_loss: 0.0167 - val_accuracy: 0.9940
Epoch 6/30
146/146 [==============================] - 1s 10ms/step - loss: 0.0138 - accuracy: 0.9970 - val_loss: 0.0147 - val_accuracy: 0.9948
Epoch 7/30
146/146 [==============================] - 2s 10ms/step - loss: 0.0117 - accuracy: 0.9981 - val_loss: 0.0115 - val_accuracy: 0.9974

In [14]:
history_cnn_2 = train_model(model_cnn_2, 30)

Epoch 1/30


C:\Users\vit\.conda\envs\HICT\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


146/146 [==============================] - 5s 29ms/step - loss: 0.0587 - accuracy: 0.9830 - val_loss: 1.4755 - val_accuracy: 0.5267
Epoch 2/30
146/146 [==============================] - 4s 30ms/step - loss: 0.0315 - accuracy: 0.9894 - val_loss: 0.7430 - val_accuracy: 0.7405
Epoch 3/30
146/146 [==============================] - 5s 31ms/step - loss: 0.0211 - accuracy: 0.9927 - val_loss: 0.0802 - val_accuracy: 0.9707
Epoch 4/30
146/146 [==============================] - 5s 32ms/step - loss: 0.0261 - accuracy: 0.9914 - val_loss: 0.0305 - val_accuracy: 0.9897
Epoch 5/30
146/146 [==============================] - 4s 30ms/step - loss: 0.0281 - accuracy: 0.9912 - val_loss: 0.0206 - val_accuracy: 0.9914
Epoch 6/30
146/146 [==============================] - 5s 31ms/step - loss: 0.0152 - accuracy: 0.9944 - val_loss: 0.0298 - val_accuracy: 0.9853
Epoch 7/30
146/146 [==============================] - 5s 31ms/step - loss: 0.0140 - accuracy: 0.9946 - val_loss: 0.0188 - val_accuracy: 0.9966
Epoch 8/30

In [15]:
history_LeNet = train_model(model_LeNet, 30)

Epoch 1/30
146/146 [==============================] - 3s 15ms/step - loss: 0.0954 - accuracy: 0.9578 - val_loss: 0.0591 - val_accuracy: 0.9767
Epoch 2/30
146/146 [==============================] - 2s 14ms/step - loss: 0.0335 - accuracy: 0.9907 - val_loss: 0.0254 - val_accuracy: 0.9948
Epoch 3/30
146/146 [==============================] - 2s 13ms/step - loss: 0.0261 - accuracy: 0.9933 - val_loss: 0.0267 - val_accuracy: 0.9948
Epoch 4/30
146/146 [==============================] - 2s 14ms/step - loss: 0.0258 - accuracy: 0.9929 - val_loss: 0.0400 - val_accuracy: 0.9871
Epoch 5/30
146/146 [==============================] - 2s 13ms/step - loss: 0.0224 - accuracy: 0.9948 - val_loss: 0.0238 - val_accuracy: 0.9922
Epoch 6/30
146/146 [==============================] - 2s 13ms/step - loss: 0.0194 - accuracy: 0.9955 - val_loss: 0.0209 - val_accuracy: 0.9940
Epoch 7/30
146/146 [==============================] - 2s 13ms/step - loss: 0.0165 - accuracy: 0.9959 - val_loss: 0.0161 - val_accuracy: 0.9940

In [56]:
model_cnn.save_weights('checkpoints/model_cnn_weights/checkpoint_cpu')
model_cnn_2.save_weights('checkpoints/model_cnn_2_weights/checkpoint_cpu')
model_LeNet.save_weights('checkpoints/model_LeNet_weights/checkpoint_cpu')

# Загрузка сохранённых моделей

In [10]:
model_cnn.load_weights('D:/Study/HICT/HICT_Patterns/checkpoints/model_cnn_weights/checkpoint_cpu')
model_cnn_2.load_weights('D:/Study/HICT/HICT_Patterns/checkpoints/model_cnn_2_weights/checkpoint_cpu')
model_LeNet.load_weights('D:/Study/HICT/HICT_Patterns/checkpoints/model_LeNet_weights/checkpoint_cpu')

# Создание и обучение моделей для определения вне главной диагонали

In [17]:
data_dir = pathlib.Path(f'{local_path}datasets/images_10')

with tf.device('/device:CPU:0'):
  train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

  val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

  AUTOTUNE = tf.data.AUTOTUNE
  train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
  val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 2732 files belonging to 2 classes.
Using 2186 files for training.
Found 2732 files belonging to 2 classes.
Using 546 files for validation.


In [18]:
model_cnn_uncentered = create_model_cnn()
model_cnn_2_uncentered = create_model_cnn_2()
model_LeNet_uncentered = create_LeNet5()

In [19]:
history_cnn_uncentered = train_model(model_cnn_uncentered, 30)

Epoch 1/30
69/69 [==============================] - 2s 10ms/step - loss: 0.0376 - accuracy: 0.9844 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/30
69/69 [==============================] - 1s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/30
69/69 [==============================] - 1s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/30
69/69 [==============================] - 1s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/30
69/69 [==============================] - 1s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/30
69/69 [==============================] - 1s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/30
69/69 [==============================] - 1s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_los

In [20]:
history_cnn_2_uncentered = train_model(model_cnn_2_uncentered, 30)

Epoch 1/30


C:\Users\vit\.conda\envs\HICT\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


69/69 [==============================] - 3s 33ms/step - loss: 0.0277 - accuracy: 0.9899 - val_loss: 0.0311 - val_accuracy: 1.0000
Epoch 2/30
69/69 [==============================] - 2s 30ms/step - loss: 7.4442e-05 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 3/30
69/69 [==============================] - 2s 31ms/step - loss: 1.8666e-04 - accuracy: 1.0000 - val_loss: 4.9005e-05 - val_accuracy: 1.0000
Epoch 4/30
69/69 [==============================] - 2s 31ms/step - loss: 3.6151e-05 - accuracy: 1.0000 - val_loss: 8.6059e-06 - val_accuracy: 1.0000
Epoch 5/30
69/69 [==============================] - 2s 30ms/step - loss: 1.9405e-05 - accuracy: 1.0000 - val_loss: 3.8959e-06 - val_accuracy: 1.0000
Epoch 6/30
69/69 [==============================] - 2s 31ms/step - loss: 1.9671e-05 - accuracy: 1.0000 - val_loss: 2.2927e-06 - val_accuracy: 1.0000
Epoch 7/30
69/69 [==============================] - 2s 32ms/step - loss: 1.2702e-05 - accuracy: 1.0000 - val_loss: 1.3497e-06 - v

In [21]:
history_LeNet_uncentered = train_model(model_LeNet_uncentered, 30)

Epoch 1/30
69/69 [==============================] - 2s 17ms/step - loss: 0.0241 - accuracy: 0.9973 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/30
69/69 [==============================] - 1s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/30
69/69 [==============================] - 1s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/30
69/69 [==============================] - 1s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/30
69/69 [==============================] - 1s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/30
69/69 [==============================] - 1s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/30
69/69 [==============================] - 1s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_l

In [ ]:
model_cnn_uncentered.save_weights('checkpoints/model_cnn_weights/checkpoint_cpu_uncentered')
model_cnn_2_uncentered.save_weights('checkpoints/model_cnn_2_weights/checkpoint_cpu_uncentered')
model_LeNet_uncentered.save_weights('checkpoints/model_LeNet_weights/checkpoint_cpu_uncentered')

# Export all models

In [12]:
model_cnn_uncentered.load_weights('D:/Study/HICT/HICT_Patterns/checkpoints/model_cnn_weights/checkpoint_cpu_uncentered')
model_cnn_2_uncentered.load_weights('D:/Study/HICT/HICT_Patterns/checkpoints/model_cnn_2_weights/checkpoint_cpu_uncentered')
model_LeNet_uncentered.load_weights('D:/Study/HICT/HICT_Patterns/checkpoints/model_LeNet_weights/checkpoint_cpu_uncentered')

In [22]:
model_cnn.save("D:/Study/HICT/HICT_Patterns/exported_models/centered_first.keras")
model_cnn_2.save("D:/Study/HICT/HICT_Patterns/exported_models/centered_second.keras")
model_LeNet.save("D:/Study/HICT/HICT_Patterns/exported_models/centered_third.keras")

model_cnn_uncentered.save("D:/Study/HICT/HICT_Patterns/exported_models/final_first.keras")
model_cnn_2_uncentered.save("D:/Study/HICT/HICT_Patterns/exported_models/final_second.keras")
model_LeNet_uncentered.save("D:/Study/HICT/HICT_Patterns/exported_models/final_third.keras")